In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
import pickle



In [3]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


def carregar_pickle(nome_arquivo):
  with open(nome_arquivo, 'rb') as arquivo:
    objeto = pickle.load(arquivo)
  return objeto

In [4]:
model2Test = keras.models.load_model('exp/bmw6/best_model_acc6_03.h5', custom_objects={"f1_m": f1_m , 'precision_m': precision_m,
        'recall_m': recall_m })

In [5]:
raw_data_set = carregar_pickle('dataset_H3_W3S1.pkl')
input_full = raw_data_set['inputs']
output_full = raw_data_set['outputs']
sequences_full = raw_data_set['sequences']

In [7]:
def getFirstNElementsWithPredictAboveOrUnderPredict(inputList, model, isAboveOrequal, predictValueRef, n):
    sList = []
    for input in inputList:
        v = model.predict(input)
        if isAboveOrequal and v >= predictValueRef:
            sList.append(input)
        elif not isAboveOrequal and v < predictValueRef:
            sList.append(input)

        if len(sList) >= n:
            break

    return sList

In [18]:
input_full[0].shape

(497, 16)

In [16]:
l = []
l.append(input_full[0])
model2Test.predict(l)

ValueError: in user code:

    File "c:\Users\pluck\anaconda3\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\pluck\anaconda3\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\pluck\anaconda3\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\pluck\anaconda3\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "c:\Users\pluck\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\pluck\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Inp

In [ ]:
goodList = getFirstNElementsWithPredictAboveOrUnderPredict(input_full, model2Test, True, 0.95, 5)

In [30]:
#len(model2Test.layers[1].get_weights()[0])

len(model2Test.layers[1].get_weights()[0][0])

16